<a href="https://colab.research.google.com/github/sachini786/FYP/blob/main/comparison_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
import pandas as pd
import numpy as np
auth.authenticate_user()

In [ ]:
%%bigquery df --project fyp-sachini
SELECT id, title, body, tags
FROM `bigquery-public-data.stackoverflow.posts_questions`
ORDER BY creation_date DESC
LIMIT 100000

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
temp_df=df.copy()

**Extract code**

In [ ]:
from bs4 import BeautifulSoup
def extract_and_remove_code_blocks(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    code_blocks = []
    for code in soup.find_all('code'):
        if code is not None and code.get_text(strip=True):
            code_blocks.append(str(code))  # Convert tag to string immediately
            code.decompose()

    cleaned_html = str(soup)
    code_blocks = ' '.join(code_blocks)
    return cleaned_html, code_blocks
temp_df[['body', 'code']] = temp_df['body'].apply(extract_and_remove_code_blocks).apply(pd.Series)

temp_df

,id,title,body,tags,code
0,73842327,Unable to start ypserv in Ubuntu,<p>I was trying to configure NIS master Server...,linux|ubuntu|linux-kernel|nis,
1,73842326,BOTH WINDOWS POWERSHELL / ISE HAS STOPPED WORK...,<p>Description:\nStopped working</p>\n<p>Probl...,powershell|visual-studio-code,
2,73842325,Migrating Java 8 project to Java 17,<p>We want to migrate our microservices which ...,java-8|migration|upgrade|java-17,
3,73842324,I'm trying to implement quickselect by partiti...,<p>Here's the java code</p>\n<p>The select fun...,java|quickselect,<code>public static Comparable select(Comparab...
4,73842323,Invalid request (Unsupported SSL request) - La...,<p>Microsoft Windows 11 Pro Build 22000\nPHP 8...,laravel|ssl,<code>D:\Programming\XAMPP\xampp\htdocs\shoppi...
...,...,...,...,...,...
99995,73613114,NuGet transitive dependencies and floating not...,<p>According to Microsoft's documentation usin...,nuget|nuget-spec,<code>MyApp\n \-- PackageA\n</code> <code>&lt...
99996,73613113,How to transform a stream of arrays to CSV in JQ?,<p>I've got this informations from a json file...,json|csv|jq,"<code>[\n ""https://localhost/rundeck/project/..."
99997,73613108,Struggling with CopyRect in Delphi 11 VCL,<p>I am struggling to get CopyRect to work wit...,delphi|bitmap,<code>procedure TForm2.Button1Click(Sender: TO...
99998,73613103,How to put one 3D object behind another in man...,"<p>When I plot overlapping 3D objects, the obj...",python|manim,<code>class TT(ThreeDScene):\n def construc...


**Extract stack trace**

In [ ]:
from bs4 import BeautifulSoup
import re
# Precompiled pattern for stack trace & exceptions
combined_trace_and_exception_pattern = re.compile(
    r'^\s*(?:'
    r'at\s+[\w.$]+\(.*\)|'
    r'Caused\s+by:.*|'
    r'\.{3}\s+\d+\s+more|'
    r'[\w$.]*Exception(?::.*)?|'
    r'[\w.$]*Error(?::.*)?|'
    r'System\.\w+Exception|'
    r'at org\.openqa\.selenium\..*\(.*\)|'
    r'File ".*", line \d+, in .+|'
    r'at .*?\(.+:\d+:\d+\)|'
    r'at .*?\(.+\) in .+:line \d+|'
    r'from .+:\d+:in .+|'
    r'#\d+ .+\(.+\): .+|'
    r'\S+\.\S+\(\S+:\d+\)|'
    r'.+ at .+ line \d+|'
    r'at [\w.$]+\(.+\.scala:\d+\)|'
    r'\*\* \(.+\) .+:\d+: .+|'
    r'at [\w.$]+\(.+\.kt:\d+\)|'
    r'thread\s\'.*\'\spanicked\s+at\s+\'.+\',\s+.+:\d+:\d+|'
    r'\d+:\s+.+\(.*\)|'
    r'.+:\s+line\s+\d+:.+|'
    r'at\s+.+,\s+.+:\s+line\s+\d+|'
    r'#\d+\s+.+ \(.+\.dart:\d+:\d+\)|'
    r'\d+\s+[\S]+\s+0x[0-9a-fA-F]+\s+\+\s+\d+|'
    r'\d+\s+[\w.]+\s+\S+\s+\+\s+\d+|'
    r'.+:\s+error:\s+.+|'
    r'panic: .+'
    r')$',
    re.MULTILINE
)

keywords = ["at ", "Exception", "Error", "File ", "panic:", "#", "Caused by:", "line "]


def extract_stack_trace_lines_and_strip_html_keep_code(html_content):
    if pd.isna(html_content) or html_content is None:
        return "", ""

    soup = BeautifulSoup(html_content, 'html.parser')
    stack_trace_lines = []

    for code_tag in soup.find_all('code'):
        code_text = code_tag.get_text()
        if not any(kw in code_text for kw in keywords):
            continue

        original_lines = code_tag.get_text().splitlines()
        cleaned_lines = []

        for line in original_lines:
            if combined_trace_and_exception_pattern.match(line):
                stack_trace_lines.append(line)
            else:
                cleaned_lines.append(line)
        code_tag.string = '\n'.join(cleaned_lines)

    return str(soup), "\n".join(stack_trace_lines)

temp_df[['code_without_stack_trace', 'stack_trace']] = (
    temp_df['code'].apply(extract_stack_trace_lines_and_strip_html_keep_code).apply(pd.Series)
)
temp_df['tags'] = temp_df['tags'].apply(lambda x: x.split('|') if isinstance(x, str) else x)

In [ ]:
temp_df.replace('', np.nan, inplace=True)

In [ ]:
temp_df.dropna(subset=['stack_trace'], inplace=True)

In [ ]:
temp_df

,id,title,body,tags,code,code_without_stack_trace,stack_trace
7,73842319,Laravel error after optimize:clear : Call to a...,"<p>After using , my whole installation is bro...",[laravel],<code>php artisan optimize:clear</code> <code>...,<code>php artisan optimize:clear</code> <code>...,#0 [laravel root folder]/vendor/laravel/frame...
28,73842285,Python: Receiving (ValueError: {error} not in ...,<p>I wanted to make a code which make a list c...,"[python, valueerror]",<code>import string\nList1= [string.digits+str...,<code>import string\nList1= [string.digits+str...,ValueError: ‘2’ is not in list
70,73842208,np.vectorize using list of argments python,<p>My input is a list of lists like this (abou...,[python],"<code>mylist = [['a', 0, 1, 0, 1], ['b', 1, 2,...","<code>mylist = [['a', 0, 1, 0, 1], ['b', 1, 2,...",ValueError: not enough values to unpack (expec...
74,73842197,Is there a way to call functions from a reload...,<p>I'm currently working on a Project and I'm ...,"[python, tensorflow]",<code>class beamModule(tf.Module):\n def __...,<code>class beamModule(tf.Module):\n def __...,AttributeError: '_UserObject' object has no at...
128,73842108,Access Gitlab container via https,<p>I had a Docker container created in Powersh...,"[docker, https, gitlab]",<code>docker run --detach \n --hostname gitla...,<code>docker run --detach \n --hostname gitla...,Error: Secure connection failed.
...,...,...,...,...,...,...,...
99915,73613308,Rails 7 without Sprockets and with Propshaft b...,<p>I tried to deploy the Rails 7 application w...,"[ruby-on-rails, sprockets, ruby-on-rails-7]",<code>Gemfile</code> <code>sprockets</code> <c...,<code>Gemfile</code> <code>sprockets</code> <c...,from /tmp/codon/tmp/buildpacks/50d5eddf222...
99919,73613296,Yocto Honister + cargo-native = missing openssl,"<p>Trying to build recipe <a href=""https://lay...","[rust, yocto, openembedded, libssl]",<code>Failed to find OpenSSL development heade...,<code>Failed to find OpenSSL development heade...,| cargo:warning=build/expando.c:4:24: error:...
99929,73613279,django.db.utils.ProgrammingError: cannot cast ...,<p>I am getting following error when trying to...,"[python, django, postgresql, heroku, deployment]",<code>django.db.utils.ProgrammingError: cannot...,"<code>LINE 1: ...ts"" ALTER COLUMN ""month"" TYPE...",django.db.utils.ProgrammingError: cannot cast ...
99941,73613253,why tkinter error rise when using turtle on py...,"<p>I'm new to Python, so I'm trying some turtl...","[python, macos, tkinter]",<code>Traceback (most recent call last):\n Fi...,<code>Traceback (most recent call last):\n ...,"File ""/Users/amerm/Desktop/CP_1/Turtle_new.p..."


In [ ]:
# Find the first non-empty stack trace
first_non_empty_stack_trace = temp_df[temp_df['stack_trace'] != ''].iloc[0]['stack_trace']
print(first_non_empty_stack_trace)

#0  [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1700): Illuminate\\Database\\Eloquent\\Model::resolveConnection(NULL)
#1  [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1492): Illuminate\\Database\\Eloquent\\Model->getConnection()
#2  [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1409): Illuminate\\Database\\Eloquent\\Model->newBaseQueryBuilder()
#3  [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1445): Illuminate\\Database\\Eloquent\\Model->newModelQuery()
#4  [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1398): Illuminate\\Database\\Eloquent\\Model->newQueryWithoutScopes()
#5  [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(2234): Illuminate\\Database\\Eloquent\\Model->newQuery()
#6  [laravel root folder]/vendor/laravel/framew

**clean code**

In [ ]:
def clean_text(html_content):
    if pd.isna(html_content) or html_content is None:
        return ""
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text()

In [ ]:
temp_df['clean_code'] = temp_df['code_without_stack_trace'].apply(clean_text)

In [ ]:
display(temp_df['clean_code'])

,clean_code
0,
1,
2,
3,public static Comparable select(Comparable[] a...
4,D:\Programming\XAMPP\xampp\htdocs\shopping-app...
...,...
99995,MyApp\n \-- PackageA\n <PackageReference Incl...
99996,"[\n ""https://localhost/rundeck/project/Enterp..."
99997,procedure TForm2.Button1Click(Sender: TObject)...
99998,class TT(ThreeDScene):\n def construct(self...


**clean titile and body**

In [ ]:
def clean_text(text):
    text = text.lower()
    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.replace('.', ' ')
    text = re.sub(r'\s+', ' ', text).strip() #replaces them with a single space.
    return text

In [ ]:
temp_df['clean_title'] = temp_df['title'].apply(clean_text)
temp_df['clean_body'] = temp_df['body'].apply(clean_text)

/tmp/ipython-input-128326439.py:3: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
display(temp_df[['clean_title', 'clean_body']])

,clean_title,clean_body
0,unable to start ypserv in ubuntu,i was trying to configure nis master server bu...
1,both windows powershell / ise has stopped work...,description: stopped working problem signature...
2,migrating java 8 project to java 17,we want to migrate our microservices which are...
3,i'm trying to implement quickselect by partiti...,here's the java code the select function retur...
4,invalid request (unsupported ssl request) - la...,microsoft windows 11 pro build 22000 php 8 1 6...
...,...,...
99995,nuget transitive dependencies and floating not...,according to microsoft's documentation using f...
99996,how to transform a stream of arrays to csv in jq?,i've got this informations from a json file ap...
99997,struggling with copyrect in delphi 11 vcl,i am struggling to get copyrect to work with d...
99998,how to put one 3d object behind another in man...,"when i plot overlapping 3d objects, the object..."


No charts were generated by quickchart


**lemmatization titile and body only**

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def advanced_clean(text):
    if pd.isna(text):
        return ""
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc if token.is_alpha and token.text not in STOP_WORDS]
    return ' '.join(lemmatized)

In [ ]:
temp_df['lemma_title'] = temp_df['clean_title'].apply(advanced_clean)


In [ ]:
display(temp_df['lemma_title'])

,lemma_title
0,unable start ypserv ubuntu
1,window powershell ise stop work
2,migrate java project java
3,try implement quickselect partition array cons...
4,invalid request unsupported ssl request laravel
...,...
99995,nuget transitive dependency float notation
99996,transform stream array csv jq
99997,struggle copyrect delphi vcl
99998,object manim show


In [ ]:
temp_df['lemma_body'] = temp_df['clean_body'].apply(advanced_clean)

In [ ]:
display(temp_df['lemma_body'])

,lemma_body
0,try configure nis master server ypserv fail st...
1,description stop work problem signature proble...
2,want migrate microservice build java framewaor...
3,java code select function return kth small ele...
4,microsoft windows pro build php laravel framew...
...,...
99995,accord microsoft documentation float notation ...
99996,get information json file aplye jq filter need...
99997,struggle copyrect work delphi vcl crop portion...
99998,plot overlap object object background visible ...


**clean stack trace**

In [ ]:
from bs4 import BeautifulSoup
import re

def clean_stack_trace(trace):
    if trace is None or (isinstance(trace, str) and trace.strip() == ''):
        return ""
    trace = BeautifulSoup(str(trace), "html.parser").get_text()
    trace = re.sub(r'[ \t]+', ' ', trace) # Replaces multiple spaces or tabs with a single space
    lines = [line.strip() for line in trace.splitlines()] #Splits the trace into lines
    trace = "\n".join(lines).strip() #Joins cleaned

    return trace

In [ ]:
temp_df['clean_stack_trace'] = temp_df['stack_trace'].apply(clean_stack_trace)

In [ ]:
first_non_empty_stack_trace = temp_df[temp_df['clean_stack_trace'] != ''].iloc[0]['clean_stack_trace']
print(first_non_empty_stack_trace)

#0 [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1700): Illuminate\\Database\\Eloquent\\Model::resolveConnection(NULL)
#1 [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1492): Illuminate\\Database\\Eloquent\\Model->getConnection()
#2 [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1409): Illuminate\\Database\\Eloquent\\Model->newBaseQueryBuilder()
#3 [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1445): Illuminate\\Database\\Eloquent\\Model->newModelQuery()
#4 [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(1398): Illuminate\\Database\\Eloquent\\Model->newQueryWithoutScopes()
#5 [laravel root folder]/vendor/laravel/framework/src/Illuminate/Database/Eloquent/Model.php(2234): Illuminate\\Database\\Eloquent\\Model->newQuery()
#6 [laravel root folder]/vendor/laravel/framework/src

In [ ]:
ERZX   .

**Feature Engineering**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

In [ ]:
for col in ['lemma_title', 'lemma_body', 'clean_code', 'clean_stack_trace']:
    temp_df[col] = temp_df[col].fillna('')

In [ ]:
tfidf_title = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_text = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
tfidf_code = TfidfVectorizer(max_features=4000, ngram_range=(1, 2))
tfidf_stack = TfidfVectorizer(max_features=4000, ngram_range=(1, 2))

In [ ]:
X_title = tfidf_title.fit_transform(temp_df['lemma_title'])
X_text = tfidf_text.fit_transform(temp_df['lemma_body'])
X_code = tfidf_code.fit_transform(temp_df['clean_code'])
X_stack = tfidf_stack.fit_transform(temp_df['clean_stack_trace'])

In [ ]:
X = hstack([X_title, X_text, X_code, X_stack])

In [ ]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(temp_df['tags'])

In [ ]:
min_len = min(X.shape[0], Y.shape[0])
X = X[:min_len]
Y = Y[:min_len]

tag_centroids = {}
for i, tag in enumerate(mlb.classes_):
    tag_docs_idx = np.where(Y[:, i] == 1)[0]
    if len(tag_docs_idx) > 0:
        tag_vector = X[tag_docs_idx].mean(axis=0)
        tag_centroids[tag] = tag_vector

In [ ]:
def recommend_tags_cosine_separate(row, top_k=5):
    vec_title = tfidf_title.transform([row["lemma_title"]])
    vec_text = tfidf_text.transform([row["lemma_body"]])
    vec_code = tfidf_code.transform([row["clean_code"]])
    vec_stack = tfidf_stack.transform([row["clean_stack_trace"]])

    vec = hstack([vec_title, vec_text, vec_code, vec_stack])
    vec_dense = np.asarray(vec.todense())

    similarities = []
    for tag, centroid in tag_centroids.items():
        centroid_dense = np.asarray(centroid)
        sim = cosine_similarity(vec_dense, centroid_dense).flatten()[0]
        similarities.append((tag, sim))

    top_tags = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    return [tag for tag, _ in top_tags]

In [ ]:
def evaluate_recommender_cosine_separate(df, top_k=5):
    total_precision = 0
    total_recall = 0
    total_samples = 0

    for _, row in df.iterrows():
        true_tags = row['tags']
        if not true_tags:
            continue

        predicted_tags = recommend_tags_cosine_separate(
            row, top_k=top_k
        )
        true_tags_set = set(true_tags)
        predicted_tags_set = set(predicted_tags)


        hits = len(predicted_tags_set & true_tags_set)
        precision = hits / top_k if top_k > 0 else 0
        recall = hits / len(true_tags_set) if len(true_tags_set) > 0 else 0

        total_precision += precision
        total_recall += recall
        total_samples += 1

    avg_precision = total_precision / total_samples if total_samples > 0 else 0
    avg_recall = total_recall / total_samples if total_samples > 0 else 0
    avg_f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0


    print(f"Precision@{top_k}: {avg_precision:.4f}")
    print(f"Recall@{top_k}: {avg_recall:.4f}")
    print(f"F1-score@{top_k}: {avg_f1:.4f}")

**evaluation metrix**

In [ ]:
evaluate_recommender_cosine_separate(temp_df.sample(n=100, random_state=39), top_k=5)

Precision@5: 0.5700
Recall@5: 0.8363
F1-score@5: 0.6779


In [ ]:
print(f"Information for post ID {post_id}:")
print("Title:", post_info['lemma_title'].iloc[0])
print("Body:", post_info['lemma_body'].iloc[0])
print("Code:", post_info['clean_code'].iloc[0])
print("Stack Trace:", post_info['clean_stack_trace'].iloc[0])
recommended_tags = recommend_tags_cosine_separate(post_info.iloc[0], top_k=5)
print("\nRecommended Tags:", recommended_tags)


Information for post ID 73619049:
Title: pyspark kafka java lang noclassdeffounderror org apache kafka common security jaascontext
Body: encounter problem print datum console kafka topic error message show image snapshot error message understand root cause error occur help follow kafka spark version code solve error
Code: 22/09/06 10:14:02 error microbatchexecution: query [id = ba6cb0ca-a3b1-41be-9551-7956650fbdab, runid = 7dfb967a-8bc2-4c73-91ae-5356cfb21ea4] terminated with error exception in thread "stream execution thread for [id = ba6cb0ca-a3b1-41be-9551-7956650fbdab, runid = 7dfb967a-8bc2-4c73-91ae-5356cfb21ea4]" java lang noclassdeffounderror: org/apache/kafka/common/security/jaascontext traceback (most recent call last): orders_agg_write_stream awaittermination() return self _jsq awaittermination() raise converted from none === streaming query === identifier: [id = ba6cb0ca-a3b1-41be-9551-7956650fbdab, runid = 7dfb967a-8bc2-4c73-91ae-5356cfb21ea4] current committed offsets: {} 

In [ ]:
import random

# Select a random post from the DataFrame
post_id = random.choice(temp_df['id'].tolist())
post_info = temp_df[temp_df['id'] == post_id]

print(f"Information for post ID {post_id}:")
print("Title:", post_info['lemma_title'].iloc[0])
print("Body:", post_info['lemma_body'].iloc[0])
print("Code:", post_info['clean_code'].iloc[0])
print("Stack Trace:", post_info['clean_stack_trace'].iloc[0])
print("\nActual Tags:", post_info['tags'].iloc[0])

recommended_tags = recommend_tags_cosine_separate(post_info.iloc[0], top_k=5)
print("\nRecommended Tags:", recommended_tags)

Information for post ID 73698651:
Title: mongoservererror bad auth authentication fail problem try connect mongodb cluster
Body: tell problem reason get authentication problem try connect mongodb atla error message get env file include link mongodb cluster rsyvyrl mongodb majority finally entry point
Code:     at TLSSocket.emit (node:events:527:28) {
  ok: 0,
  code: 8000,
  codeName: 'AtlasError',
  [Symbol(errorLabels)]: Set(1) { 'HandshakeError' }
}
[nodemon] app crashed - waiting for file changes before starting... db.js    const mongoose = require('mongoose')

const connectDB = async () => {
  try {
    const conn = await mongoose.connect(process.env.MONGO_URI)

    console.log(`MongoDB Connected: ${conn.connection.host}`.cyan.underline)
  } catch (error) {
    console.log(error)
    process.exit(1)
  }
}


module.exports = connectDB server.js const express = require('express')
const colors = require('colors');
const connectDB = require('./config/db')
const dotenv = require('doten

In [ ]:
def recommend_tags_cosine_without_stacktrace(row, top_k=5):
    vec_title = tfidf_title.transform([row["lemma_title"]])
    vec_text = tfidf_text.transform([row["lemma_body"]])
    vec_code = tfidf_code.transform([row["clean_code"]])

    vec = hstack([vec_title, vec_text, vec_code])
    vec_dense = np.asarray(vec.todense())

    similarities = []
    for tag, centroid in tag_centroids_without_stacktrace.items():
        sim = cosine_similarity(vec_dense, centroid.reshape(1, -1)).flatten()[0]
        similarities.append((tag, sim))

    top_tags = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    return [tag for tag, _ in top_tags]

    tag_centroids_without_stacktrace = compute_tag_centroids_without_stacktrace()

In [ ]:
def evaluate_recommender_cosine_separate(df, recommender_func, top_k=5):
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    total_samples = 0

    for _, row in df.iterrows():
        true_tags = row['tags']
        if not true_tags:
            continue

        predicted_tags = recommender_func(row, top_k=top_k)
        hits = len(set(predicted_tags) & set(true_tags))
        precision = hits / top_k
        recall = hits / len(true_tags)
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        total_precision += precision
        total_recall += recall
        total_f1 += f1
        total_samples += 1

    avg_precision = total_precision / total_samples
    avg_recall = total_recall / total_samples
    avg_f1 = total_f1 / total_samples

    print(f"Precision@{top_k}: {avg_precision:.4f}")
    print(f"Recall@{top_k}: {avg_recall:.4f}")
    print(f"F1-score@{top_k}: {avg_f1:.4f}")

In [ ]:
evaluate_recommender_cosine_separate(temp_df.sample(n=100, random_state=42),recommender_func=recommend_tags_cosine_without_stacktrace,top_k=5)

Precision@5: 0.5160
Recall@5: 0.7867
F1-score@5: 0.6018
